In [1]:
# Importando os pacotes necessários
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

In [2]:
# Comandos auxiliares
from IPython.core.display import display, HTML
pd.set_option('display.max_columns', 10)
pd.options.mode.chained_assignment = None  # default='warn'

def display_side_by_side(dfs: list, captions: list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes(
            "style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

# Transformação de Dados
---

Em geral, após um processo de <ins>**Integração de dados**</ins>, colunas com diferentes tipos de dados podem ser geradas. 

Na <ins>**Transformação de dados**</ins>, o principal objetivo é exatamente **transformar** esses dados de diferentes formatos em um formato suportado pelo processo de pesquisa, e.g., modelos e algoritmos. 

A <ins>**Transformação de dados**</ins> possui algumas etapas em comum com a <ins>**Limpeza de dados**</ins>, incluindo técnicas de remoção de ruído e discretização de dados. 

Portanto, nesta etapa, nós vamos focar nas técnicas que ainda não foram abordadas até aqui.

## Dados não padronizados

Dados coletados de diferentes fontes podem reunir dados **heterogêneos**, **não padronizados** e em **diferentes escalas** que muitas vezes necessitam um pré-processamento, i.e., uma etapa de escalonamento. 

O escalonamento de tais dados não é uma etapa obrigatória, mas uma boa prática! 

> **Mas... Por que precisamos dimensionar as variáveis em nosso conjunto de dados?** 🤔

> *Muitos algoritmos de Aprendizado de Máquina apresentam um melhor desempenho quando as variáveis numéricas de entrada são padronizadas.*

Isso inclui algoritmos que usam uma soma ponderada da entrada como, por exemplo, a regressão linhear e algoritmos que usam medidas de distância, como os algoritmos K-means e K-NN.

Existem algumas técnicas de escalonamento que **transformam** características para uma escala, magnitude ou intervalo. 

Para descrever e exemplicar cada uma delas, utilizaremos o conjunto de atributos numéricos que descrevem as músicas do Spotify, ou seja, a tabela `Tracks` do nosso conjunto de dados. 

In [3]:
# Lendo os dados
data = pd.read_table('../dataset/spotify_hits_dataset_complete.tsv', encoding='utf-8')
data.head(3)

,song_id,song_name,artist_id,artist_name,popularity,...,liveness,loudness,speechiness,valence,tempo
0,2rRJrJEo19S2J82BDsQ3F7,Falling,['7uaIm6Pw7xplS8Dy06V6pT'],['Trevor Daniel'],77,...,0.0887,-8.756,0.0364,0.236,127.087
1,3BYIzNZ3t9lRQCACXSMLrT,Venetia,['4O15NlyKLIASxsJ0PrXPfz'],['Lil Uzi Vert'],66,...,0.1480,-4.139,0.1750,0.562,142.933
2,1g3J9W88hTG173ySZR6E9S,Tilidin Weg,['1aS5tqEs9ci5P9KD9tZWa6'],['Bonez MC'],13,...,0.0838,-8.544,0.2330,0.171,109.090


In [4]:
# Selecionando apenas as variáveis numéricas
df = data.select_dtypes(include=['number'])
# print(df.columns.values) # imprime as colunas restantes

Depois de selecionar as variáveis numéricas da nossa tabela, ficamos com as seguintes informações:

* `popularity`: *score* de popularidade da track (o valor varia entre 0 e 100, sendo 100 o mais popular)
* `track_number`: número da track no álbum que ela pertence
* `num_artists`: número total de intérpretes da track
* `num_available_markets`: número total de países nos quais a track pode ser reproduzida
* `duration_ms`: duração da track em milissegundos
* [`key` - `tempo`]: características acústicas da track

Utilizando esses atributos, nós iremos explorar dois tipos de escalonamento: *Normalização* e *Padronização*.

### NORMALIZAÇÃO

A *Normalização* é uma técnica de escalonamento em que os valores são **deslocados** e **redimensionados** para que fiquem entre 0 e 1. Essa técnica também é conhecida como nomarlização *Min-Max* e é usada para **transformar** dados em uma <ins>escala semelhante</ins>.

A fórmula da *Normalização* é dada por:

$$X' = \frac{X - X_{min}}{X_{max} - X_{min}},$$

onde $X_{max}$ e $X_{min}$ são os valores máximo e mínimo do atributo, respectivamente.

**RESUMINDO**
* O valor mínimo da coluna será transformado em 0
* O valor máximo da coluna será transformado em 1
* Os valores que estiverem entre o valor mínimo e máximo, estarão entre o intervalo de 0 a 1

#### EXEMPLO
No exemplo a seguir, nós iremos normalizar todas as colunas numéricas da tabela `Tracks`.

Isto é, para cada atributo (i.e., coluna), nós aplicaremos a fórmula da *Normalização Min-Max*. 

Para isso, nós utilizaremos os métodos `min()` e `max()` do *pandas*:

In [5]:
df_norm = df.copy()  # cópia do DataFrame

# Para cada coluna de df_norm,
for coluna in df_norm.columns:
    
    # Normalização Min-Max
    X = df_norm[[coluna]]
    df_norm[[coluna]] = (X - X.min()) / (X.max() - X.min())

display_side_by_side([df.head(3), df_norm.head(3)], ['Original', 'Normalizado']) # imprime as 3 primeiras linhas

,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,77,10,1,177,159381,10,0,4,0.123000,0.784000,0.430000,0.000000,0.088700,-8.756000,0.036400,0.236000,127.087000
1,66,14,1,178,188800,9,1,4,0.162000,0.775000,0.757000,0.000000,0.148000,-4.139000,0.175000,0.562000,142.933000
2,13,1,1,0,180950,10,0,4,0.113000,0.882000,0.530000,0.479000,0.083800,-8.544000,0.233000,0.171000,109.090000
,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,0.793814,0.310345,0.000000,0.994382,0.284679,0.909091,0.000000,0.750000,0.123557,0.769231,0.424552,0.000000,0.073225,0.615183,0.015335,0.212314,0.506887
1,0.680412,0.448276,0.000000,1.000000,0.349477,0.818182,1.000000,0.750000,0.162801,0.757692,0.773018,0.000000,0.136156,0.788266,0.176348,0.558386,0.606828
2,0.134021,0.000000,0.000000,0.000000,0.332187,0.909091,0.000000,0.750000,0.113495,0.894872,0.531117,0.502623,0.068025,0.623130,0.243727,0.143312,0.393380


### PADRONIZAÇÃO

A *Padronização* é outra técnica de escalonamento em que os valores são **centralizados** em torno da média. Isso significa que a média de cada atributo se iguala a 0 e a distribuição resultante tem um desvio padrão igual a 1.

A fórmula da *Padronização* é dada por:

$$X' = \frac{X - \mu}{\sigma},$$

onde $\mu$ e $\sigma$ são a média e o desvio padrão dos valores do atributo, respectivamente.

#### EXEMPLO
No exemplo a seguir, nós iremos padronizar todas as colunas numéricas da tabela `Tracks`.

Isto é, para cada coluna, os valores serão subtraídos pela média e divididos pelo desvio padrão, aplicando a fórmula descrita anteriormente. Para isso, nós utilizaremos os métodos `mean()` e `std()` do *pandas*:

In [6]:
df_padron = df.copy()  # cópia do DataFrame

# Para cada coluna de df_padron,
for coluna in df_padron.columns:
    
    # Padronização
    X = df_padron[[coluna]]
    df_padron[[coluna]] = (X - X.mean()) / (X.std())

display_side_by_side([df.head(3), df_padron.head(3)], ['Original', 'Padronizado']) # imprime as 3 primeiras linhas

,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,77,10,1,177,159381,10,0,4,0.123000,0.784000,0.430000,0.000000,0.088700,-8.756000,0.036400,0.236000,127.087000
1,66,14,1,178,188800,9,1,4,0.162000,0.775000,0.757000,0.000000,0.148000,-4.139000,0.175000,0.562000,142.933000
2,13,1,1,0,180950,10,0,4,0.113000,0.882000,0.530000,0.479000,0.083800,-8.544000,0.233000,0.171000,109.090000
,popularity,track_number,num_artists,num_available_markets,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,0.448125,1.007112,-0.644346,0.361651,-0.846963,1.277028,-1.138397,0.106209,-0.483188,0.621100,-1.231196,-0.160381,-0.640537,-0.896190,-0.802165,-1.217354,0.122480
1,-0.164943,1.806576,-0.644346,0.380038,-0.181157,1.003471,0.877744,0.106209,-0.327328,0.556130,0.809644,-0.160381,-0.225699,0.871711,0.462704,0.221431,0.658761
2,-3.118819,-0.791681,-0.644346,-2.892824,-0.358817,1.277028,-1.138397,0.106209,-0.523152,1.328550,-0.607086,6.303556,-0.674816,-0.815013,0.992015,-1.504228,-0.486598


---
## Engenharia de características (Feature Engineering)

Além do escalonamento, outra forma de **transformar** os dados é através da **Engenharia de características**. Nesta técnica, características de um conjunto de dados podem ser criadas ou removidas para melhorar o desempenho de modelos de Aprendizado de Máquina. 

Muitas técnicas de <ins>**Transformação**</ins> e <ins>**Limpeza de dados**</ins> também podem ser consideradas **Engenharia de características**, incluindo métodos de escalonamento, discretização e redução de dimensionalidade. 

Portanto, a seguir, nós vamos discutir algumas técnicas que ainda não foram abordadas.

#### EXEMPLO

Para exemplificar, utilizaremos dessa vez as variáveis não-numéricas da tabela `Tracks`.

In [7]:
# Selecionando apenas variáveis não-numéricas
df = data.select_dtypes(exclude=['number'])
df = df.drop(columns=['song_id', 'song_name', 'artist_id', 'artist_name']) # removendo colunas desnecessárias
# print(df.columns.values) # imprime as colunas restantes

Depois de selecionar as variáveis, ficamos com os seguintes dados:

* `explicit`: *flag* indicando se a track contém conteúdo explícito
* `song_type`: se a track é do tipo *Solo* ou *Collaboration*
* `release_date`: data de lançamento da track

Utilizando esses atributos, nós iremos explorar duas formas de **Engenharia de características**: 

1. Converter variáveis categóricas em numéricas
2. Criar novos atributos a partir de variáveis de data

### CONVERTER DADOS CATEGÓRICOS

Variáveis categóricas são atributos que só podem assumir um número **limitado** e, geralmente, **fixo** de valores possíveis. 

Por exemplo, as músicas do Spotify contêm dados categóricos para informar se as músicas são explícitas ou não (`explicit`). 

In [8]:
df[['explicit']].head(3)

,explicit
0,False
1,True
2,False


Ou seja, cada unidade de observação (i.e., música) é atribuída a um determinado grupo ou categoria nominal (i.e. [`False` ou `True`]). 

No entanto, muitos algoritmos e modelos de Aprendizado de Máquina têm dificuldades em processar dados categóricos, necessitando alguma forma de **transformar** tais atributos em variáveis numéricas. 

Uma das abordagens mais simples e comuns é converter variáveis categóricas em *dummies* ou indicadores, utilizando a função `get_dummies()` do *pandas*, conforme o exemplo a seguir. 

In [9]:
# Selecionando os atributos categóricos
df = data[['explicit', 'song_type']].astype('category')

# Convertendo em dummies
df_dummies = pd.get_dummies(df)

display_side_by_side([df.head(3), df_dummies.head(3)], ['Original', 'Transformado']) # imprime as 3 primeiras linhas

O método `get_dummies()` cria uma nova coluna contendo 0s e 1s para cada categoria possível do atributo, onde:

* `0`: indica que o dado não é daquela categoria
* `1`: indica que o dado é daquela categoria 

Como os dois atributos têm apenas duas categorias possíveis, o método criou duas colunas para cada atributo.


> Bom, agora que vimos como converter **dados categóricos** em dados **numéricos**, vamos ver como criar novos atributos a partir de variáveis de **data**! 😉

### VARIÁVEIS DE DATA

Datas e horários são fontes valiosas de informações que podem ser usadas em projetos de Ciência de Dados.

Por exemplo, em um modelo de predição de sucesso musical, pode ser útil considerar a data de lançamento das músicas. De fato, existem casos onde o período de lançamento influencia diretamente no sucesso musical.
**Exemplos:** músicas de Carnaval, músicas natalinas, músicas da Copa, etc.
 
No entanto, tecnicamente, as variáveis de data requerem alguma **Engenharia de características** para **transformá-las** em dados numéricos antes de serem usadas pelos algoritmos de Aprendizado de Máquina. 

#### EXEMPLO

Para exemplificar, utilizaremos a data de lançamento das músicas do Spotify (`release_date`) convertendo em formatos mais amigáveis, extraindo recursos e criando novas variáveis que podem ser usadas na análise de algum modelo. 

In [10]:
# Selecionando o id, nome e a data de lançamento das tracks
df = data[['song_id', 'song_name', 'release_date']]
df.head()

,song_id,song_name,release_date
0,2rRJrJEo19S2J82BDsQ3F7,Falling,2020-03-26
1,3BYIzNZ3t9lRQCACXSMLrT,Venetia,2020-03-06
2,1g3J9W88hTG173ySZR6E9S,Tilidin Weg,2020-07-30
3,75pQqzwgCjUOSSy5CpmAjy,Pero Ya No,2020-02-28
4,7kDUspsoYfLkWnZR7qwHZl,my ex's best friend (with blackbear),2020-09-25


O primeiro passo do processamento de datas é verificar se as variáveis estão em formato `datetime`.
Para isso, podemos utilizar o `dtypes` para verificar o tipo de cada coluna do nosso *DataFrame*: 

In [11]:
df.dtypes # verificando o tipo das colunas de df

song_id         object
song_name       object
release_date    object
dtype: object

Como podemos notar, o tipo da variável `release_date` é `object` e não `datetime`. Portanto, precisamos convertê-la antes de passar para a **Engenharia de características**. Para isso, usaremos o método `to_datetime()` do *pandas*:

In [12]:
df['release_date'] = pd.to_datetime(df['release_date']) # convertendo a data de lançamento em 'datetime'

In [13]:
df.dtypes # verificando o tipo das colunas de df

song_id                 object
song_name               object
release_date    datetime64[ns]
dtype: object

**Deu certo!** 😃

A biblioteca *pandas* disponibiliza diversas propriedades para variáveis `datetime`, incluindo:

* `year`: o ano da `datetime`
* `month`: o mês da `datetime`, onde `January=1` e `December=12`
* `day`: o dia da `datetime`

Então, depois de converter nossa variável para o tipo `datetime`, nós podemos quebrar a data de lançamento das músicas utilizando essas três propriedades:

In [14]:
# Extraindo propriedades básicas de uma variável datetime
df['day'] = df['release_date'].dt.day  # dia
df['month'] = df['release_date'].dt.month  # mês
df['year'] = df['release_date'].dt.year  # ano
df.head(3)

,song_id,song_name,release_date,day,month,year
0,2rRJrJEo19S2J82BDsQ3F7,Falling,2020-03-26,26,3,2020
1,3BYIzNZ3t9lRQCACXSMLrT,Venetia,2020-03-06,6,3,2020
2,1g3J9W88hTG173ySZR6E9S,Tilidin Weg,2020-07-30,30,7,2020


Além dessas propriedades básicas, existem outras informações que podemos extrair da nossa variável `datetime`.

Por exemplo, a semana do ano, o dia da semana e o trimestre:

In [16]:
# Extraindo propriedades extras de uma variável datetime
df['week'] = df['release_date'].dt.isocalendar().week  # semana do ano
df['dayofweek_index'] = df['release_date'].dt.dayofweek  # dia da semana (índice)
df['dayofweek_name'] = df['release_date'].dt.day_name() # dia da semana (nome)
df['quarter'] = df['release_date'].dt.quarter # trimestre
df.head(3)

,song_id,song_name,release_date,day,month,year,week,dayofweek_index,dayofweek_name,quarter
0,2rRJrJEo19S2J82BDsQ3F7,Falling,2020-03-26,26,3,2020,13,3,Thursday,1
1,3BYIzNZ3t9lRQCACXSMLrT,Venetia,2020-03-06,6,3,2020,10,4,Friday,1
2,1g3J9W88hTG173ySZR6E9S,Tilidin Weg,2020-07-30,30,7,2020,31,3,Thursday,3


## Conclusão

Este notebook apresentou como fazer a transformação de dados de diferentes formatos a partir de diversas abordagens.

🔎 **Se interessou?** 

* Para mais propriedades de variáveis `datetime`, veja a documentação do *pandas*: [DatetimeIndex](https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.html)

---

O próximo notebook ([2.3.Ciencia.Dados.Basica.ipynb](2.3.Ciencia.Dados.Basica.ipynb)) apresenta métodos para realiar a Análise de Exploração de Dados.